In [6]:
!pip install gradio
!pip install pypdf2
!pip install langchain
!pip install langchain-google-genai
!pip install python-dotenv
!pip install faiss-cpu


In [7]:
import os
os.environ["GOOGLE_API_KEY"] = "########"

In [8]:
!pip install langchain-community # Install the missing langchain-community module

In [9]:
import gradio as gr
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        with open(pdf.name, "rb") as f:
            pdf_reader = PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=1000,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context. If the answer is not in the provided context, just say, "answer is not available in the context". Do not provide a wrong answer.

    Context:\n {context}\n
    Question:\n {question}\n
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    return response.get("output_text", "No answer found.")

def process_pdfs(pdf_docs):
    raw_text = get_pdf_text(pdf_docs)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)
    return "PDF files processed successfully!"

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Chat with Multiple PDFs using Gemini")
    gr.Markdown("Upload your PDF files, process them, and ask questions to get detailed answers!")

    with gr.Tab("Upload PDFs"):
        pdf_docs = gr.Files(label="Choose PDF files")  # Corrected to use `gr.Files` without `type` parameter
        process_button = gr.Button("Process PDFs")
        process_output = gr.Textbox(label="Processing Status")
        process_button.click(fn=process_pdfs, inputs=pdf_docs, outputs=process_output)

    with gr.Tab("Ask a Question"):
        user_question = gr.Textbox(label="Ask a Question from the PDF Files", lines=2)
        answer_button = gr.Button("Get Answer")
        answer_output = gr.Textbox(label="Answer", lines=4)
        answer_button.click(fn=user_input, inputs=user_question, outputs=answer_output)

if __name__ == "__main__":
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://330548b1cc9dd7c773.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
